### ✅ Fórmulas reales de SSB:

Dada una señal real $m(t)$ y su transformada de Hilbert $\hat{m}(t)$:

* **USB (Upper Sideband):**

$$
s_{\text{USB}}(t) = m(t) \cdot \cos(2\pi f_c t) - \hat{m}(t) \cdot \sin(2\pi f_c t)
$$

* **LSB (Lower Sideband):**

$$
s_{\text{LSB}}(t) = m(t) \cdot \cos(2\pi f_c t) + \hat{m}(t) \cdot \sin(2\pi f_c t)
$$

> La única diferencia entre ambas es el signo delante del término con $\hat{m}(t)$.

---

### ✅ Fórmula para SSB con Portadora (SSB-FC)

$$
s_{SSB-FC}(t) = A_c \cdot \cos(2\pi f_c t) + m(t) \cdot \cos(2\pi f_c t) \pm \hat{m}(t) \cdot \sin(2\pi f_c t)
$$

Donde:

* $A_c \cdot \cos(2\pi f_c t)$: es la **portadora transmitida**.
* $m(t)$: señal de mensaje.
* $\hat{m}(t)$: transformada de Hilbert de $m(t)$.
* El signo **+** para **USB**, y **−** para **LSB**.

---

### 🧠 En otras palabras:

Es simplemente:

$$
s_{SSB-FC}(t) = A_c \cdot \cos(2\pi f_c t) + s_{SSB-SC}(t)
$$

---

### 📌 Ejemplo en palabras:

Si estás transmitiendo **solo USB con portadora**, la fórmula sería:

$$
s_{USB-FC}(t) = A_c \cdot \cos(2\pi f_c t) + m(t) \cdot \cos(2\pi f_c t) + \hat{m}(t) \cdot \sin(2\pi f_c t)
$$

Y para **LSB con portadora**:

$$
s_{LSB-FC}(t) = A_c \cdot \cos(2\pi f_c t) + m(t) \cdot \cos(2\pi f_c t) - \hat{m}(t) \cdot \sin(2\pi f_c t)
$$


In [69]:
import os
import subprocess
import sys

import soundfile as sf
import sounddevice as sd
import numpy as np
import matplotlib.pyplot as plt

from scipy.signal import butter, filtfilt
from scipy.signal import hilbert, welch

# from src.script_plot import ModuladorDSBSC

In [48]:
# import os
# import soundfile as sf

class GetAudio:
    def get_wav_info(self, ruta_archivo):
        with sf.SoundFile(ruta_archivo) as f:
            self.audio_data = f.read(dtype='float32')
            self.audio_samplerate = f.samplerate
            self.audio_n_canales = f.channels
            # self.audio_n_frames = len(f)
            # self.audio_duration = self.audio_n_frames / self.audio_samplerate
            # self.audio_subtype = f.subtype
            self.audio_filename = os.path.basename(ruta_archivo)

audio = GetAudio()
# audio.get_wav_info("../utils/audio_estereo.wav")
audio.get_wav_info("../utils/vowel_3.wav")
# 
print(audio.audio_filename)
print(audio.audio_samplerate)
print(audio.audio_data.shape[0])
print(audio.audio_n_canales)
sd.play(audio.audio_data, audio.audio_samplerate)
sd.wait()


vowel_3.wav
24000
6891
1


# Clase SSB

### 🔍 ¿Qué está pasando aquí?

1. `audio_data` es tu señal de mensaje $m(t)$: real, por ejemplo una señal de voz.

2. `hilbert(audio_data)` devuelve una **señal analítica** compleja:

   $$
   \text{analytic\_signal}(t) = m(t) + j \cdot \hat{m}(t)
   $$

   * Parte real → $m(t)$
   * Parte imaginaria → $\hat{m}(t)$, la transformada de Hilbert de $m(t)$

3. Entonces:

   * `m = np.real(...)` → tu señal original $m(t)$
   * `m_hat = np.imag(...)` → transformada de Hilbert $\hat{m}(t)$

📌 Nota: Si el `audio_data` ya es una señal real, **la parte real del resultado es idéntica** a `audio_data`. Puedes incluso omitir `np.real` y usar directamente `audio_data`.

---

## 🎚️ Luego haces esto:

```python
cos_carrier = np.cos(2 * np.pi * frecuencia_carrier * time)
sin_carrier = np.sin(2 * np.pi * frecuencia_carrier * time)

usb = m * cos_carrier - m_hat * sin_carrier
lsb = m * cos_carrier + m_hat * sin_carrier
```

### 🔍 ¿Qué significa esto?

Está aplicando la fórmula real de la modulación SSB:

$$
s_{\text{SSB}}(t) = m(t) \cos(2\pi f_c t) \mp \hat{m}(t) \sin(2\pi f_c t)
$$

* El signo **menos** (−) genera la **banda lateral superior (USB)**.
* El signo **más** (+) genera la **banda lateral inferior (LSB)**.

✅ ¡Este es el enfoque correcto y clásico para SSB usando Hilbert!

---

### 💥 Entonces, ¿cuál es tu duda exactamente?

Si estás confundido por esto:

> ¿Por qué se usa la parte real e imaginaria del resultado de `hilbert`?

Es porque:

* La **parte real** es la señal original $m(t)$
* La **parte imaginaria** es la **transformada de Hilbert** $\hat{m}(t)$, la que se necesita para construir la señal SSB

---

### 🧠 ¿Y por qué usar la señal analítica?

Porque **la modulación SSB** se basa en usar:

$$
s(t) = \text{Re}\{[m(t) + j \hat{m}(t)] \cdot e^{j2\pi f_c t}\}
$$

Y si desarrollas eso usando identidades trigonométricas, obtienes:

$$
s(t) = m(t)\cos(2\pi f_c t) - \hat{m}(t)\sin(2\pi f_c t)
$$

(para USB).

---

## ✅ Resumen final

| Concepto                 | Código                                  | Significado             |
| ------------------------ | --------------------------------------- | ----------------------- |
| Señal original (mensaje) | `m = np.real(analytic_signal)`          | $m(t)$                  |
| Transformada de Hilbert  | `m_hat = np.imag(analytic_signal)`      | $\hat{m}(t)$            |
| Señal analítica          | `analytic_signal = hilbert(audio_data)` | $m(t) + j \hat{m}(t)$   |
| Carrier coseno           | `cos(2πf_ct)`                           | Multiplica $m(t)$       |
| Carrier seno             | `sin(2πf_ct)`                           | Multiplica $\hat{m}(t)$ |
| USB                      | `m*cos - m_hat*sin`                     | Banda lateral superior  |
| LSB                      | `m*cos + m_hat*sin`                     | Banda lateral inferior  |


In [ ]:
# from scipy.signal import butter, filtfilt
# from scipy.signal import hilbert, welch

# frecuencia de portadora ?
# error de frecuencia ?
# error de fase ?
# archivo ?

class SSB:
    # constructor
    def __init__(self):
        pass

    def ssb_mono_mod(self, audio_data, audio_samplerate, frecuencia_carrier, sc_or_fc, nombre_banda):
        sd.play(audio_data, audio_samplerate)
        sd.wait()

        N = len(audio_data)
        print(f"{N}")
        time = np.linspace(0, N / audio_samplerate, N, endpoint=False)
        print(f"Longitud de time: {len(time)}")

        analytic_signal = hilbert(audio_data)
        mensaje = np.real(analytic_signal)
        mensaje_hat = np.imag(analytic_signal)

        # Carriers
        cos_carrier = np.cos(2 * np.pi * frecuencia_carrier * time)
        sin_carrier = np.sin(2 * np.pi * frecuencia_carrier * time)

        # --- Generar bandas laterales ---
        usb = mensaje * cos_carrier - mensaje_hat * sin_carrier  # USB
        lsb = mensaje * cos_carrier + mensaje_hat * sin_carrier  # LSB

        # --- Reproducción de señales moduladas ---
        print("Reproduciendo USB ")
        # sd.play(usb / np.max(np.abs(usb)), samplerate=audio_samplerate)
        sd.play(usb, audio_samplerate)
        sd.wait()

        print("Reproduciendo LSB ")
        # sd.play(lsb / np.max(np.abs(lsb)), samplerate=audio_samplerate)
        sd.play(lsb, audio_samplerate)
        sd.wait()

        # subprocess.Popen([
        #     sys.executable, "script_plot.py",
        #     str(500),
        #     str(frecuencia_carrier),
        #     "espectro"  # o "senal", o "ambos"
        # ])
        
        return usb, lsb
    

    def ssb_mono_demod(self, nombre_banda, banda_lateral, frecuencia_carrier, sample_rate):
        
        # --- Filtro pasa bajas ---
        def lowpass(signal, fs, cutoff=4000, order=6):  # cutoff según el ancho de banda del audio
            nyq = fs / 2
            b, a = butter(order, cutoff / nyq, btype='low')
            return filtfilt(b, a, signal)

        N = len(banda_lateral)
        # print(f"{N}")
        time = np.linspace(0, N / sample_rate, N, endpoint=False)

        # --- Demodulación coherente USB ---
        banda_lateral = banda_lateral * 2 * np.cos(2 * np.pi * frecuencia_carrier * t)  # recuperar m(t)
        banda_lateral_filtered = lowpass(banda_lateral, sample_rate)

        # --- Demodulación coherente LSB ---
        # demod_lsb = lsb * 2 * np.cos(2 * np.pi * fc * t)
        # demod_lsb_filtered = lowpass(demod_lsb, fs)

        # --- Reproducción de audio demodulado ---
        print(f"Reproduciendo audio demodulado desde {nombre_banda}")
        sd.play(banda_lateral_filtered / np.max(np.abs(banda_lateral_filtered)), sample_rate)
        sd.wait()

        # print("Reproduciendo audio demodulado desde LSB...")
        # sd.play(demod_lsb_filtered / np.max(np.abs(demod_lsb_filtered)), fs)
        # sd.wait()

        return banda_lateral_filtered


        


ssb = SSB()
ssb_mod = ssb.ssb_mono_mod(audio.audio_data, audio.audio_samplerate, 1000, "sc", "usb")

np.save("usb_signal.npy", ssb_mod[0])  # Guarda en disco


subprocess.Popen([
            sys.executable, "script_plot.py",
            "usb_signal.npy",
            str(audio.audio_samplerate),
            "senal"  # espectro, "senal", o "ambos"
        ])

ssb_demod = ssb.ssb_mono_demod("usb", ssb_mod[0], 1000, audio.audio_samplerate)

# print(f"ssb{ssb}")
# N = len(audio.audio_data)
# print(f"{N}")

# t = np.linspace(0, N / audio.audio_samplerate, N, endpoint=False)
# print(f"Longitud de t: {len(t)}")



6891
Longitud de time: 6891
Reproduciendo USB 
Reproduciendo LSB 
Reproduciendo audio demodulado desde usb


Figure(1000x400)
